# 추론 시간 재기(Inference Time)

## openvino모델의 추론시간(Geti)

In [1]:
!python -m pip install --upgrade pip

In [2]:
!pip install openvino

In [3]:
!python -c "from openvino.runtime import Core"

In [4]:
from openvino.runtime import Core
from pathlib import Path

model_path = "openvino.xml"
ie = Core() # initialize inference engine
network = ie.read_model(model=model_path, weights=Path(model_path).with_suffix('.bin'))
executable_network = ie.compile_model(model=network, device_name="CPU")

In [5]:
import numpy as np

im = np.random.randn(1,3,224, 224) #random input
output_layer = next(iter(executable_network.outputs)) # OpenVINO model의 output layer를 가져옴
y = executable_network([im])[output_layer] # Inference 실행하여 output_layer에 해당하는 output을 y에 할당

In [20]:
import time

start = time.time()
y = executable_network([im]) [output_layer]
print(f'time lapse: {time.time()-start}')

time lapse: 0.03080296516418457


## onnx모델의 추론시간(Geti)

In [7]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 59.7 MB/s eta 0:00:00


In [8]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 544.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 50.7 MB/s eta 0:00:00


In [9]:
%pip install -q "openvino>=2023.1.0" "nncf>=2.6.0"

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

#### onnx모델을 openvino로 변환

In [10]:
import onnx
import openvino as ov

onnx_path = "model.onnx"
ir_path = "model.xml"

ov_model = ov.convert_model(onnx_path)
ov.save_model(ov_model, ir_path)

#### onnx모델이 한 프레임을 추론하는 시간

In [22]:
import cv2
import numpy as np
import onnxruntime
import time

# Load the ONNX model
onnx_model_path = "model.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# Get the name of the input node
input_name = ort_session.get_inputs()[0].name

# Open the video file
cap = cv2.VideoCapture('input_video.avi')

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret:
        # Preprocess the frame to match the model's input shape
        frame = cv2.resize(frame, (224, 224))  # 예: 모델이 224x224 크기의 이미지를 입력으로 받는 경우
        input_data = np.array(frame).astype('float32')

        # 모델의 입력 차원에 맞게 배열을 재구성합니다.
        # 예: 모델이 배치 크기 x 채널 수 x 높이 x 너비 형식의 입력을 받는 경우
        input_data = input_data.transpose((2, 0, 1))  # HWC to CHW
        input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

        # Measure the inference time
        start_time = time.time()

        # Perform inference using the ONNX model
        output = ort_session.run(None, {input_name: input_data})

        inference_time = time.time() - start_time
        print("Inference Time:", inference_time)
    else:
        break

cap.release()

Inference Time: 0.046550750732421875
Inference Time: 0.04324650764465332
Inference Time: 0.04508686065673828
Inference Time: 0.04114651679992676
Inference Time: 0.04578685760498047
Inference Time: 0.0456695556640625
Inference Time: 0.041831016540527344
Inference Time: 0.04325556755065918
Inference Time: 0.04779672622680664
Inference Time: 0.043607234954833984
Inference Time: 0.04755234718322754
Inference Time: 0.044660091400146484
Inference Time: 0.04666900634765625
Inference Time: 0.043031930923461914
Inference Time: 0.042578697204589844
Inference Time: 0.04341435432434082
Inference Time: 0.05155587196350098
Inference Time: 0.04413557052612305
Inference Time: 0.043572425842285156
Inference Time: 0.04198598861694336
Inference Time: 0.04214811325073242
Inference Time: 0.04273176193237305
Inference Time: 0.04200005531311035
Inference Time: 0.043015480041503906
Inference Time: 0.04578661918640137
Inference Time: 0.0432438850402832
Inference Time: 0.04209113121032715
Inference Time: 0.0413

#### onnx모델이 한 동영상을 추론하는 총 시간

In [12]:
!pip install collection

  Preparing metadata (setup.py) ... done
  Created wheel for collection: filename=collection-0.1.6-py3-none-any.whl size=5099 sha256=15b7d4640d8b79a1102726d301f38431d0d162ed57bb09a295fd6788ba6f210c
  Stored in directory: /root/.cache/pip/wheels/a5/70/eb/1d28795e9384ab3b9be6359bdde9e1652f6e7dab9d26844f70
Successfully built collection


In [24]:
import cv2
import numpy as np
import onnxruntime
import time
from collections import Counter

# Load the ONNX model
onnx_model_path = "model.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# Get the name of the input node
input_name = ort_session.get_inputs()[0].name

# Open the video file
cap = cv2.VideoCapture('input_video.avi')

# List to store inference results
results = []

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret:
        # Preprocess the frame to match the model's input shape
        frame = cv2.resize(frame, (224, 224))  # 예: 모델이 224x224 크기의 이미지를 입력으로 받는 경우
        input_data = np.array(frame).astype('float32')

        # 모델의 입력 차원에 맞게 배열을 재구성합니다.
        # 예: 모델이 배치 크기 x 채널 수 x 높이 x 너비 형식의 입력을 받는 경우
        input_data = input_data.transpose((2, 0, 1))  # HWC to CHW
        input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

        # Perform inference using the ONNX model
        output = ort_session.run(None, {input_name: input_data})

        # Store the result
        results.append(tuple(output[0].flatten().tolist()))

    else:
        break

cap.release()

# Find the most common result
counter = Counter(results)
most_common_result = counter.most_common(1)[0][0]

print("Most Common Result:", most_common_result)

# Measure the time to the most common result
start_time = time.time()
while most_common_result not in results:
    time.sleep(0.01)
time_to_most_common = time.time() - start_time

print("Time to Most Common Result:", time_to_most_common)

Most Common Result: (0.23193198442459106, -0.4557848870754242, 0.22414630651474)
Time to Most Common Result: 9.72747802734375e-05


## pytorch모델의 추론시간(Geti)

In [25]:
!pip install torch torchvision

In [26]:
import torch
import numpy as np
import time

# Load the PyTorch model
model_path = "weights.pth"
model = torch.load(model_path)
model.eval()  # Set the model to evaluation mode

# Create a random input
im = torch.randn(1, 3, 224, 224)

# Measure the inference time
start = time.time()
y = model(im)
print(f'time lapse: {time.time()-start}')

RuntimeError: ignored